# E-Commerce Sales Optimization

#### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

#### Loading both datasets

In [ ]:
# Load the first CSV file
df1 = pd.read_csv('online_retail_I.csv')
print("Successfully loaded file1")

# Load the second CSV file
df2 = pd.read_csv('online_retail_II.csv')
print("Successfully loaded file2")

#### Merging both datasets into one csv file

In [ ]:
merged_df = pd.concat([df1, df2], ignore_index=True)

#### Saving merged dataset

In [ ]:
merged_df.to_csv('merged_file.csv', index=False)

print("\nFiles merged successfully!")
print("The merged data is saved in 'merged_file.csv'")

In [ ]:
merged_df['Customer ID'].value_counts()

### Basic Feature Engineering

In [ ]:
merged_df.shape

In [ ]:
merged_df.info()

In [ ]:
merged_df.describe()

In [ ]:
merged_df.isnull().sum()

In [ ]:
merged_df.columns=merged_df.columns.str.strip()

print("--- Before Processing ---")
print(merged_df.head())

print(f"\nTotal missing Customer IDs before: {merged_df['Customer ID'].isnull().sum()}")

# 1. Identify unique invoices that have missing CustomerIDs.
invoices_with_missing_ids = merged_df[merged_df['Customer ID'].isnull()]['Invoice'].unique()
print(f"\nFound {len(invoices_with_missing_ids)} unique invoices that need a new Customer ID.")

# 2. Find the maximum existing CustomerID to ensure new IDs don't conflict.
max_existing_id = int(merged_df['Customer ID'].dropna().max())
print(f"Maximum existing Customer ID is: {max_existing_id}. New IDs will start after this number.")

# 3. Create a mapping from each unique invoice to a new, unique CustomerID.
start_new_id_from = max_existing_id + 1
invoice_to_new_id_map = {
    invoice: start_new_id_from + i
    for i, invoice in enumerate(invoices_with_missing_ids)
}

# 4. Fill the missing 'Customer ID' values using the map.
merged_df['Customer ID'] = merged_df['Customer ID'].fillna(merged_df['Invoice'].map(invoice_to_new_id_map))

# 5. Convert the 'Customer ID' column to an integer type.
merged_df['Customer ID'] = merged_df['Customer ID'].astype(int)

print("\n--- After Processing ---")
print(merged_df.head())

print(f"\nTotal missing Customer IDs after: {merged_df['Customer ID'].isnull().sum()}")

merged_df.to_csv('cleaned_01.csv', index=False)
print('File Saved Successfully')

In [ ]:
new_df=pd.read_csv('cleaned_01.csv')
new_df.isnull().sum()

In [ ]:
cleaned_df=new_df.dropna(subset=['Description'])

# Save this new, smaller DataFrame to 'cleaned_02.csv'
cleaned_df.to_csv('final_data.csv', index=False)

print('Data cleaned and saved as new file')

In [ ]:
final_df=pd.read_csv('final_data.csv')
final_df.isnull().sum()

In [ ]:
final_df['Customer ID'].value_counts()

In [ ]:
final_df['Invoice'].value_counts()

In [ ]:
final_df['Description'].value_counts()

In [ ]:
final_df['Country'].value_counts()

#### Creating new feature 'TotalPrice'

In [ ]:
final_df['TotalPrice']=(final_df['Quantity']*final_df['Price']).round(2)

final_df

### EDA

#### Sales by Hour of the Day

In [ ]:
final_df['InvoiceDate']=pd.to_datetime(final_df['InvoiceDate'], format="%d-%m-%Y %H:%M")

# Extract the hour from the InvoiceDate
final_df['Hour']=final_df['InvoiceDate'].dt.hour

# Group by hour and count the number of transactions (invoices)
hourly_sales=final_df.groupby('Hour')['Invoice'].nunique().reset_index()

# Create the line plot
plt.figure(figsize=(14,7))
sns.lineplot(x='Hour',y='Invoice',data=hourly_sales,marker='o',color='indigo')
plt.title('Number of Transactions by Hour of the Day',fontsize=16)
plt.xlabel('Hour of the Day',fontsize=12)
plt.ylabel('Number of Transactions',fontsize=12)
plt.xticks(range(0,24))
plt.grid(True)
plt.show()

In [ ]:
hourly_sales=final_df.groupby('Hour')['Invoice'].nunique()
hourly_sales

#### Analysis of Order Value

In [ ]:
sales_df=final_df[final_df['TotalPrice']>0]

# Group by invoice to analyze each transaction
order_value=sales_df.groupby('Invoice').agg(
    OrderValue=('TotalPrice','sum'),
    ItemsInOrder=('Quantity','sum')
                        ).reset_index()

# Calculate and print the Average Order Value 
avg_order_value=order_value['OrderValue'].mean()
print(f"The Average Order Value for actual sales is: £{avg_order_value:.2f}")

# Plot the distribution of order values
plt.figure(figsize=(12,7))
sns.histplot(order_value[order_value['OrderValue']<1000]['OrderValue'],bins=100,kde=True,color='teal')
plt.title('Distribution of Order Values (Under £1000)',fontsize=16)
plt.xlabel('Order Value (£)',fontsize=12)
plt.ylabel('Frequency',fontsize=12)
plt.show()

In [ ]:
order_value=sales_df.groupby('Invoice').agg(
    OrderValue=('TotalPrice','sum'),
    ItemsInOrder=('Quantity','sum')
                        ).reset_index()
order_value

In [ ]:
fig = px.histogram(
    data_frame=order_value[order_value['OrderValue'] < 1000],
    x='OrderValue',
    nbins=100,
    title='Distribution of Order Values (Under £1000)',
    labels={'OrderValue': 'Order Value (£)', 'Invoice Count': 'Frequency'},
    color_discrete_sequence=['teal']
)

fig.update_layout(
    plot_bgcolor='white'
)
fig.show()

#### Relationship Between Price and Quantity

In [ ]:
final_df['Price']=pd.to_numeric(final_df['Price'],errors='coerce')
final_df['Quantity']=pd.to_numeric(final_df['Quantity'],errors='coerce')

plot_df=final_df[(final_df['Quantity']>0)&(final_df['Price']>0)].copy()

p_99=plot_df['Price'].quantile(0.99)
q_99=plot_df['Quantity'].quantile(0.99)
plot_df=plot_df[(plot_df['Price']<p_99)&(plot_df['Quantity']<q_99)]

fig = px.scatter(
    data_frame=plot_df,
    x='Price',
    y='Quantity',
    title='Price vs. Quantity Sold (Interactive)',
    labels={'Price':'Price (£)','Quantity':'Quantity Sold'},
    opacity=0.5 
)


fig.update_layout(
    title_font_size=16,
    xaxis_title_font_size=12,
    yaxis_title_font_size=12,
    plot_bgcolor='white' 
)

fig.show()

#### New vs. Returning Customer Analysis

In [ ]:
final_df['FirstPurchaseDate']=final_df.groupby('Customer ID')['InvoiceDate'].transform('min')

final_df['InvoiceMonth']=final_df['InvoiceDate'].dt.to_period('M')
final_df['FirstPurchaseMonth']=final_df['FirstPurchaseDate'].dt.to_period('M')

final_df['CustomerType']='Returning'
final_df.loc[final_df['InvoiceMonth']==final_df['FirstPurchaseMonth'],'CustomerType']='New'

cust_trend = final_df.groupby(['InvoiceMonth','CustomerType'])['Customer ID'].nunique().reset_index()
cust_trend_pivot = cust_trend.pivot(index='InvoiceMonth',columns='CustomerType',values='Customer ID').fillna(0)

cust_trend_pivot.plot(
    kind='bar',
    stacked=False, 
    figsize=(14, 8),
    colormap='viridis'
)

plt.title('New vs. Returning Customers Over Time', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of Unique Customers', fontsize=12)
plt.xticks(rotation=45, ha='right') 
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Country wise analysis

In [ ]:
sns.set_style("whitegrid")

country_revenue = final_df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(12, 8))
sns.barplot(x=country_revenue.values, y=country_revenue.index, 
            palette='viridis',hue=country_revenue.index,legend=False)

plt.xscale('log')

plt.title('Top 10 Countries by Total Revenue (Log Scale)', fontsize=16)
plt.xlabel('Total Revenue (£) - Log Scale', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.show()

#### Best selling products

In [ ]:
# Group data by product description and sum the quantity
product_quantity = final_df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)

# Create the bar plot
plt.figure(figsize=(12, 8))
sns.barplot(x=product_quantity.values,y=product_quantity.index, 
            palette='plasma',hue=product_quantity.index,legend=False)
plt.title('Top 10 Best-Selling Products by Quantity', fontsize=16)
plt.xlabel('Total Quantity Sold', fontsize=12)
plt.ylabel('Product Description', fontsize=12)
plt.show()

#### Seasonal Pattern in Sales

In [ ]:
# Create a 'YearMonth' column for grouping
final_df['YearMonth']=final_df['InvoiceDate'].dt.to_period('M')
monthly_sales=final_df.groupby('YearMonth')['TotalPrice'].sum().reset_index()

# Convert 'YearMonth' back to a plottable format
monthly_sales['YearMonth']=monthly_sales['YearMonth'].dt.to_timestamp()

# Create the line plot
plt.figure(figsize=(14, 7))
sns.lineplot(x='YearMonth', y='TotalPrice', data=monthly_sales, marker='o')
plt.title('Total Sales Revenue Over Time', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Revenue (£)', fontsize=12)
plt.grid(True)
plt.show()

#### Top 25 Customers by Sales

In [ ]:
# Group by Customer ID and sum TotalPrice
top_cust=final_df.groupby('Customer ID')['TotalPrice'].sum().sort_values(ascending=False).head(25)

# Create the bar plot
plt.figure(figsize=(14, 8))
sns.barplot(x=top_cust.values, y=top_cust.index.astype(str), 
            palette='viridis', hue=top_cust.index,legend=False)
plt.title('Top 25 Custmers by Revenue', fontsize=14)
plt.xlabel('Total Revenue (£)', fontsize=10)
plt.ylabel('Customer ID', fontsize=10)
plt.show()

#### Time Between Purchases (Purchase Frequency)

In [ ]:
# list of unique purchase dates for each cust
cust_buy=final_df[['Customer ID','InvoiceDate']].drop_duplicates()

# Sort by customer and date
cust_buy=cust_buy.sort_values(['Customer ID','InvoiceDate'])

# Calc the diff btw consecutive purchases for each cust
cust_buy['TimeDiff']=cust_buy.groupby('Customer ID')['InvoiceDate'].diff()

# Convert the time diff to days
cust_buy['DaysDiff']=cust_buy['TimeDiff'].dt.days

# Calc and print the average time
avg_time=cust_buy['DaysDiff'].mean()
print(f"Average time between customer purchases: {avg_time:.2f} days")

# Plot the distribution
plt.figure(figsize=(12, 7))
sns.histplot(cust_buy['DaysDiff'].dropna(),bins=50,kde=True, color='red')
plt.title('Distribution of Days Between Consecutive Purchases', fontsize=16)
plt.xlabel('Days Between Purchases', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xlim(0, 200) 
plt.show()

#### Product Portfolio Analysis (ABC Analysis)

In [ ]:
prod_rev=final_df.groupby('Description')['TotalPrice'].sum().sort_values(ascending=False).reset_index()

# Calculate cumulative revenue and percentage
prod_rev['CumulativeRevenue']=prod_rev['TotalPrice'].cumsum()
prod_rev['CumulativePercentage']=100 * prod_rev['CumulativeRevenue']/prod_rev['TotalPrice'].sum()

# Define ABC categories
def categorize_product(percentage):
    if percentage <= 80:
        return 'Top 80%'
    elif 80 < percentage <= 95:
        return 'Next 15%'
    else:
        return 'Bottom 5%'

prod_rev['Category']=prod_rev['CumulativePercentage'].apply(categorize_product)

# Count the number of products in each category
cat_counts = prod_rev['Category'].value_counts().reindex(['Top 80%','Next 15%','Bottom 5%'])
print("Product Count by Category:")
print(cat_counts)

# Plot the result
plt.figure(figsize=(10, 6))
sns.barplot(x=cat_counts.index, y=cat_counts.values, 
            palette='YlGnBu',hue=cat_counts.index,legend=False)
plt.title('Product Category based on Revenue', fontsize=16)
plt.xlabel('Product Category', fontsize=12)
plt.ylabel('Count of Products', fontsize=12)
plt.show()

#### Geographic Analysis (Beyond the UK)

In [ ]:
# Separate UK from the rest of the world
uk_df=final_df[final_df['Country']=='United Kingdom']
non_uk_df=final_df[final_df['Country']!='United Kingdom']

# Plot top 10 non-UK countries by revenue
top_non_uk=non_uk_df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(12, 7))
sns.barplot(x=top_non_uk.values, y=top_non_uk.index, 
            palette='cividis',hue=top_non_uk.index,legend=False)
plt.title('Top 10 Non-UK Countries by Revenue',fontsize=16)
plt.xlabel('Total Revenue (£)',fontsize=12)
plt.ylabel('Country',fontsize=12)
plt.show()

# Compare Average Order Value (AOV)
uk_aov=uk_df.groupby('Invoice')['TotalPrice'].sum().mean()
non_uk_aov=non_uk_df.groupby('Invoice')['TotalPrice'].sum().mean()

print(f"Average Order Value (UK): £{uk_aov:.2f}")
print(f"Average Order Value (Others): £{non_uk_aov:.2f}")

### Key Findings
- Business Core: The business is heavily reliant on the domestic UK market, which accounts for the vast majority of revenue. The top international markets are Netherlands, EIRE, Germany, and France.

- Sales Cycle: Sales are highly seasonal, with a significant peak in October and November leading into the holidays. The busiest time of day is between 10 AM and 3 PM.

- Customer Profile: The customer base consists of a growing number of returning customers, and a small number of VIP customers are responsible for a large portion of revenue. The average order value is approximately £523.30.

- Product Catalog: The business model is driven by high-volume sales of low-price items, with most products costing less than £5.

In [ ]:
# Saving all the work
final_df.to_csv('analytics_ready_data.csv', index=False)
print("Final DataFrame is saved")

## RFM Analysis

In [ ]:
data=pd.read_csv('analytics_ready_data.csv')
data

#### Calculating RFM Metrics

In [ ]:
import datetime as dt

# To calculate Recency, we need a "snapshot" date. This will be the day after the last transaction.
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
snapshot_date=data['InvoiceDate'].max()+dt.timedelta(days=1)

# Group by each customer and calculate RFM values
rfm_df=data.groupby('Customer ID').agg({
    'InvoiceDate': lambda date: (snapshot_date - date.max()).days, # Recency
    'Invoice': 'nunique',                                          # Frequency
    'TotalPrice': 'sum'                                            # Monetary
})

# Rename the columns to be more descriptive
rfm_df.rename(columns={'InvoiceDate': 'Recency',
                       'Invoice': 'Frequency',
                       'TotalPrice': 'MonetaryValue'}, inplace=True)

print("\nRFM Metrics Calculated:")
print(rfm_df.head())

In [ ]:
rfm_df

#### Creating RFM Scores

In [ ]:
r_labels=range(5, 0, -1) # 5 is best (most recent), 1 is worst
f_labels=range(1, 6)     # 5 is best (most frequent), 1 is worst
m_labels=range(1, 6)     # 5 is best (highest value), 1 is worst

# The .rank(method='first') ensures each value gets a unique rank, which qcut can divide perfectly.
rfm_df['R_Score']=pd.qcut(rfm_df['Recency'].rank(method='first'),q=5,labels=r_labels)
rfm_df['F_Score']=pd.qcut(rfm_df['Frequency'].rank(method='first'),q=5,labels=f_labels)
rfm_df['M_Score']=pd.qcut(rfm_df['MonetaryValue'].rank(method='first'),q=5, labels=m_labels)

print("\nRFM Scores Created:")
print(rfm_df[['Recency','R_Score','Frequency','F_Score','MonetaryValue','M_Score']].head())

In [ ]:
rfm_df

#### Defining RFM Segments

In [ ]:
rfm_df['RFM_Segment_Score']=rfm_df.apply(lambda row: str(row['R_Score'])+str(row['F_Score'])+str(row['M_Score']),axis=1)

rfm_df['RFM_Score']=rfm_df[['R_Score','F_Score','M_Score']].sum(axis=1)

# Mapping RFM scores to segment names
def assign_segment(score):
    if score >= 14:
        return 'Champions'
    elif score >= 11:
        return 'Loyal Customers'
    elif score >= 8:
        return 'Potential Loyalists'
    elif score >= 6:
        return 'At-Risk Customers'
    elif score >= 4:
        return 'Needs Attention'
    else:
        return 'Lost'
    
rfm_df['Segment']=rfm_df['RFM_Score'].apply(assign_segment)

print("\nCustomer Segments Assigned:")
print(rfm_df.head())

In [ ]:
rfm_df

In [ ]:
rfm_df['Segment'].value_counts()

In [ ]:
rfm_df.to_csv('rfm_segmented_customers.csv')

In [ ]:
rfm_df

#### Top 10 products from each Segment

In [ ]:
df_with_segments=pd.merge(final_df, rfm_df, on='Customer ID')
def get_top_products_for_segment(segment_name,n=10):
    """
    Returns the top N products for a given customer segment.
    """
    segment_df=df_with_segments[df_with_segments['Segment']==segment_name]
    top_products=segment_df['Description'].value_counts().head(n)
    return top_products

# Best customers
print("--- Top 10 Products for 'Champions' ---")
print(get_top_products_for_segment('Champions'))

print("\n" + "="*50 + "\n")
print("--- Top 10 Products for 'Loyal Customers' ---")
print(get_top_products_for_segment('Loyal Customers'))

print("\n" + "="*50 + "\n")
print("--- Top 10 Products for 'Potential Loyalists' ---")
print(get_top_products_for_segment('Potential Loyalists'))

print("\n" + "="*50 + "\n")
# Customers we might be about to lose
print("--- Top 10 Products for 'At-Risk Customers' ---")
print(get_top_products_for_segment('At-Risk Customers'))

# Customers we had lost
print("\n" + "="*50 + "\n")
print("--- Top 10 Products for 'Lost' ---")
print(get_top_products_for_segment('Lost'))

#### Top 10 Customers from each Segment

In [ ]:
df_with_segments=pd.merge(final_df, rfm_df, on='Customer ID')
def get_top_cust_for_segment(segment_name,n=10):
    """
    Returns the top N products for a given customer segment.
    """
    segment_df=df_with_segments[df_with_segments['Segment']==segment_name]
    top_products=segment_df['Customer ID'].value_counts().head(n)
    return top_products

# Best customers
print("--- Top 10 Customer for 'Champions' ---")
print(get_top_cust_for_segment('Champions'))

print("\n" + "="*50 + "\n")
print("--- Top 10 Customer for 'Loyal Customers' ---")
print(get_top_cust_for_segment('Loyal Customers'))

print("\n" + "="*50 + "\n")
print("--- Top 10 Customer for 'Potential Loyalists' ---")
print(get_top_cust_for_segment('Potential Loyalists'))

print("\n" + "="*50 + "\n")
# Customers we might be about to lose
print("--- Top 10 Customer for 'At-Risk Customers' ---")
print(get_top_cust_for_segment('At-Risk Customers'))

# Customers we had lost
print("\n" + "="*50 + "\n")
print("--- Top 10 Customer for 'Lost' ---")
print(get_top_cust_for_segment('Lost'))